<a href="https://colab.research.google.com/github/AldoFerreira17/Rede_Neural_MLP_Autoencoder/blob/main/Rede_neural_MLP_e_autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importação das Bibliotecas

Nesta célula, importamos as bibliotecas necessárias. Isso inclui TensorFlow/Keras para construir o modelo, Matplotlib para visualização e IPython para exibir as imagens dinamicamente no Colab.

In [ ]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Dense, Input, Flatten, Reshape, LeakyReLU as LR, Activation, Dropout
from tensorflow.keras.models import Model, Sequential
from matplotlib import pyplot as plt
from IPython import display
import numpy as np

Carregamento e Normalização dos Dados

Carregamos o conjunto de dados MNIST, que contém imagens de dígitos escritos à mão. Em seguida, normalizamos os dados dividindo cada valor por 255 para que os valores dos pixels fiquem entre 0 e 1.

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0

Visualização de um Exemplo de Imagem

Mostramos uma imagem do conjunto de treinamento para visualizar os dados com os quais estamos trabalhando. Esta célula ajuda a confirmar que os dados foram carregados corretamente.

In [ ]:
plt.imshow(x_train[0], cmap = "gray")
plt.show()

Definição do Codificador (Encoder)

Definimos o codificador, que é a parte do modelo responsável por comprimir as imagens em um vetor latente de dimensão menor (LATENT_SIZE). Ele usa camadas densas (fully connected) com Leaky ReLU e dropout para prevenir overfitting.

In [ ]:
LATENT_SIZE = 32
encoder = Sequential([
    Flatten(input_shape = (28, 28)),
    Dense(512),
    LR(),
    Dropout(0.5),
    Dense(256),
    LR(),
    Dropout(0.5),
    Dense(128),
    LR(),
    Dropout(0.5),
    Dense(64),
    LR(),
    Dropout(0.5),
    Dense(LATENT_SIZE),
    LR()
])

Definição do Decodificador (Decoder)

Definimos o decodificador, que reconstrói as imagens a partir do vetor latente. Ele usa a mesma arquitetura com camadas densas e Leaky ReLU, mas termina com uma ativação sigmoide e uma camada de reshaping para transformar o vetor de volta em uma imagem 28x28.


In [ ]:
decoder = Sequential([
    Dense(64, input_shape = (LATENT_SIZE,)),
    LR(),
    Dropout(0.5),
    Dense(128),
    LR(),
    Dropout(0.5),
    Dense(256),
    LR(),
    Dropout(0.5),
    Dense(512),
    LR(),
    Dropout(0.5),
    Dense(784),
    Activation("sigmoid"),
    Reshape((28, 28))
])

Definição e Compilação do Modelo

Conectamos o codificador e o decodificador em um modelo completo. A imagem de entrada é codificada no vetor latente e, em seguida, reconstruída. O modelo é compilado com o otimizador Nadam e a função de perda binary_crossentropy.

In [ ]:
img = Input(shape = (28, 28))
latent_vector = encoder(img)
output = decoder(latent_vector)
model = Model(inputs = img, outputs = output)
model.compile("nadam", loss = "binary_crossentropy")

Treinamento do Modelo com Visualização das Saídas

O modelo é treinado por várias épocas (EPOCHS). A cada época, o modelo gera uma visualização de imagens reconstruídas a partir do conjunto de teste, para monitorar visualmente o progresso do treinamento. Em seguida, ele treina com os dados de treinamento.

In [ ]:
EPOCHS = 60
for epoch in range(EPOCHS):
    fig, axs = plt.subplots(4, 4)
    rand = x_test[np.random.randint(0, 10000, 16)].reshape((4, 4, 1, 28, 28))

    display.clear_output()

    for i in range(4):
        for j in range(4):
            axs[i, j].imshow(model.predict(rand[i, j])[0], cmap = "gray")
            axs[i, j].axis("off")

    plt.subplots_adjust(wspace = 0, hspace = 0)
    plt.show()
    print("-----------", "EPOCH", epoch, "-----------")
    model.fit(x_train, x_train)